In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();


In [2]:
import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

import quantstats as qs

# extend pandas functionality with metrics, etc.
qs.extend_pandas()

In [3]:
df = pd.DataFrame()
conn = MySQLdb.connect(host='tokenmetrics.cluster-cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
cursor = conn.cursor()
    
query = '''SELECT * FROM ico_ml_grade_history;'''
data = pd.read_sql_query(query, conn)
df = pd.concat([df,data])

conn.close()

In [4]:
def minmax_scaling(array, columns, min_val=0, max_val=1):
    """
    array : pandas DataFrame or NumPy ndarray, shape = [n_rows, n_columns].
    columns : array-like, shape = [n_columns]
        Array-like with column names, e.g., ['col1', 'col2', ...]
        or column indices [0, 2, 4, ...]
    min_val : `int` or `float`, optional (default=`0`)
        minimum value after rescaling.
    max_val : `int` or `float`, optional (default=`1`)
        maximum value after rescaling.
    """
    ary_new = array.astype(float)
    if len(ary_new.shape) == 1:
        ary_new = ary_new[:, np.newaxis]

    if isinstance(ary_new, pd.DataFrame):
        ary_newt = ary_new.loc
    elif isinstance(ary_new, np.ndarray):
        ary_newt = ary_new
    else:
        raise AttributeError('Input array must be a pandas'
                             'DataFrame or NumPy array')

    numerator = ary_newt[:, columns] - ary_newt[:, columns].min(axis=0)
    denominator = (ary_newt[:, columns].max(axis=0) -
                   ary_newt[:, columns].min(axis=0))
    ary_newt[:, columns] = numerator / denominator

    if not min_val == 0 and not max_val == 1:
        ary_newt[:, columns] = (ary_newt[:, columns] *
                                (max_val - min_val) + min_val)

    return ary_newt[:, columns]

In [11]:
temp = df[df['ico_id'] == ico]

In [14]:
temp['final_grade'].min().min()

54.8020439148

In [5]:
percentile = pd.DataFrame()

for ico in df['ico_id'].unique().tolist():
    temp = df[df['ico_id'] == ico]
    temp = temp.sort_values('date', ascending=False)

    res = minmax_scaling(temp[['final_grade']], columns=['final_grade'], min_val=temp['final_grade'].min().min(), max_val=99)
    final = res.iloc[0][0]

    res = minmax_scaling(temp[['fundamental_grade']], columns=['fundamental_grade'], min_val=temp['fundamental_grade'].min().min(), max_val=99)
    fund = res.iloc[0][0]

    res = minmax_scaling(temp[['technology_grade']], columns=['technology_grade'], min_val=temp['technology_grade'].min().min(), max_val=99)
    tech = res.iloc[0][0]

    res = minmax_scaling(temp[['technical_grade']], columns=['technical_grade'], min_val=temp['technical_grade'].min().min(), max_val=99)
    technical = res.iloc[0][0]

    temp = pd.DataFrame([ico,final,fund,tech,technical])
    percentile = pd.concat([percentile, temp.T], axis=0)

In [6]:
percentile.columns = ['ico_id','final_grade','fundamental_grade','technology_grade','technical_grade']
percentile.set_index('ico_id', inplace=True)

In [7]:
percentile

,final_grade,fundamental_grade,technology_grade,technical_grade
ico_id,,,,
2974.0,72.087741,80.651822,61.427498,75.506054
3060.0,77.009971,79.289509,70.973634,69.603494
3078.0,48.467106,64.619809,51.750565,49.090477
3108.0,99.000000,95.497091,61.651582,99.000000
3110.0,58.235822,32.816410,66.736893,85.431337
...,...,...,...,...
3788.0,99.000000,42.846420,74.143773,99.000000
3791.0,23.395733,29.415887,65.181842,43.076923
3792.0,51.026688,54.654467,18.933132,62.329785


In [8]:
percentile.T.to_dict()

{2974.0: {'final_grade': 72.08774102292475,
  'fundamental_grade': 80.65182158526491,
  'technology_grade': 61.4274978638,
  'technical_grade': 75.50605383085272},
 3060.0: {'final_grade': 77.00997146376014,
  'fundamental_grade': 79.28950856441531,
  'technology_grade': 70.97363388308294,
  'technical_grade': 69.60349418641852},
 3078.0: {'final_grade': 48.4671058655,
  'fundamental_grade': 64.61980913596088,
  'technology_grade': 51.7505645752,
  'technical_grade': 49.090477278878836},
 3108.0: {'final_grade': 99.0,
  'fundamental_grade': 95.49709084557742,
  'technology_grade': 61.65158152113599,
  'technical_grade': 99.0},
 3110.0: {'final_grade': 58.23582154126867,
  'fundamental_grade': 32.8164100647,
  'technology_grade': 66.73689278422279,
  'technical_grade': 85.43133698201221},
 3119.0: {'final_grade': 79.24233989633535,
  'fundamental_grade': 80.94037858110065,
  'technology_grade': 62.4793395996,
  'technical_grade': 96.20698320121747},
 3120.0: {'final_grade': 64.110394488

In [5]:
temp = df[df['ico_id'] == 2974]

In [16]:
temp.dtypes

id                            int64
ico_id                        int64
date                         object
final_grade                 float64
fundamental_grade           float64
technology_grade            float64
technical_grade             float64
created_at           datetime64[ns]
updated_at           datetime64[ns]
dtype: object

In [34]:
temp = temp.sort_values('date', ascending=False)

In [35]:
temp

,id,ico_id,date,final_grade,fundamental_grade,technology_grade,technical_grade,created_at,updated_at
1176,1440306,2974,2020-01-01,57.736813,44.951561,70.964188,57.307693,2020-01-01 06:40:01,2020-01-01 06:40:01
1069,1431639,2974,2019-12-31,57.736813,44.951561,70.964188,57.307693,2019-12-31 23:55:00,2019-12-31 23:55:00
963,1315195,2974,2019-12-30,58.539131,47.921299,73.324287,54.494381,2019-12-30 23:59:00,2019-12-30 23:59:00
855,1161517,2974,2019-12-29,50.445061,48.138794,65.993263,37.592594,2019-12-29 23:59:00,2019-12-29 23:59:00
777,1038645,2974,2019-12-28,62.554829,63.587215,75.361488,47.346367,2019-12-28 23:59:00,2019-12-28 23:59:00
641,886914,2974,2019-12-27,52.156704,56.034515,66.310158,47.238373,2019-12-27 23:59:00,2019-12-27 23:59:00
534,732834,2974,2019-12-26,57.350170,55.360977,70.521606,44.512196,2019-12-26 23:59:00,2019-12-26 23:59:00
427,583676,2974,2019-12-25,57.526321,60.920681,65.585587,46.409573,2019-12-25 23:59:00,2019-12-25 23:59:00
320,429596,2974,2019-12-24,57.757690,53.577026,72.506889,47.500000,2019-12-24 23:59:00,2019-12-24 23:59:00
213,277763,2974,2019-12-23,54.141052,65.122185,67.030304,43.218086,2019-12-23 23:59:00,2019-12-23 23:59:00


In [57]:
res = minmax_scaling(temp[['final_grade']], columns=['final_grade'], min_val=temp['final_grade'].min().min(), max_val=99)

In [60]:
res.iloc[0][0]

79.68183647435013

In [48]:
temp

,id,ico_id,date,final_grade,fundamental_grade,technology_grade,technical_grade,created_at,updated_at
1176,1440306,2974,2020-01-01,57.736813,44.951561,70.964188,57.307693,2020-01-01 06:40:01,2020-01-01 06:40:01
1069,1431639,2974,2019-12-31,57.736813,44.951561,70.964188,57.307693,2019-12-31 23:55:00,2019-12-31 23:55:00
963,1315195,2974,2019-12-30,58.539131,47.921299,73.324287,54.494381,2019-12-30 23:59:00,2019-12-30 23:59:00
855,1161517,2974,2019-12-29,50.445061,48.138794,65.993263,37.592594,2019-12-29 23:59:00,2019-12-29 23:59:00
777,1038645,2974,2019-12-28,62.554829,63.587215,75.361488,47.346367,2019-12-28 23:59:00,2019-12-28 23:59:00
641,886914,2974,2019-12-27,52.156704,56.034515,66.310158,47.238373,2019-12-27 23:59:00,2019-12-27 23:59:00
534,732834,2974,2019-12-26,57.350170,55.360977,70.521606,44.512196,2019-12-26 23:59:00,2019-12-26 23:59:00
427,583676,2974,2019-12-25,57.526321,60.920681,65.585587,46.409573,2019-12-25 23:59:00,2019-12-25 23:59:00
320,429596,2974,2019-12-24,57.757690,53.577026,72.506889,47.500000,2019-12-24 23:59:00,2019-12-24 23:59:00
213,277763,2974,2019-12-23,54.141052,65.122185,67.030304,43.218086,2019-12-23 23:59:00,2019-12-23 23:59:00


In [56]:
temp['final_grade'].min().min()

50.44506073

In [49]:
s1 = pd.Series([1, 2, 3, 4, 5, 6], index=(range(6)))
s2 = pd.Series([10, 9, 8, 7, 6, 5], index=(range(6)))
df = pd.DataFrame(s1, columns=['s1'])
df['s2'] = s2
df

,s1,s2
0,1,10
1,2,9
2,3,8
3,4,7
4,5,6
5,6,5


In [52]:
minmax_scaling(df, columns=['s1', 's2'], min_val=30,max_val=100)


,s1,s2
0,30.0,100.0
1,44.0,86.0
2,58.0,72.0
3,72.0,58.0
4,86.0,44.0
5,100.0,30.0
